# Source detection

In [17]:
import subprocess
# import multiprocess
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy import wcs
import os
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
from astropy.convolution import Gaussian2DKernel, convolve_fft
from astropy.visualization import make_lupton_rgb
import warnings
warnings.filterwarnings("ignore")

In [18]:
plt.style.use("default")
plt.rc('xtick', direction='in', top=True)
plt.rc('ytick', direction='in', right=True)
plt.rc('axes', linewidth=1.15)

plt.rc("mathtext", fontset="dejavuserif")

In [19]:
if not os.path.exists('../Data/Source_cat'):
    os.system('mkdir ../Data/Source_cat')

if not os.path.exists('../Data/Source_cat/BG_maps'):
    os.system('mkdir ../Data/Source_cat/BG_maps')

image_file = "../Data/Images/merged_image_200_2300.fits"
expmap_file = "../Data/Images/merged_expmap_200_2300.fits"

In [20]:
LOG_file = open("../Data/Source_cat/process.log", "w+")

## ERMASK:
It generates a detection mask for the eSASS source detection chain. The resulting product is a FITS image with values 0 and 1, where 1 indicates the image area on which the subsequent source detection tasks will be executed.

In [21]:
def run_ermask(exposure_map, output_mask_file, log_file=None):
    subprocess.run(["ermask", 
                    f"expimage={exposure_map}", 
                    f"detmask={output_mask_file}",
                    ], stdout=log_file, stderr=log_file)

In [22]:
output_mask_file = "../Data/Source_cat/detmask.fits"

if os.path.exists(output_mask_file):
    os.remove(output_mask_file)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_ermask(expmap_file, output_mask_file, log_file=log_file)

## ERBOX (local):
This step performs sliding box source detection in local mode. The objective of the local detection step is to create an initial list of source positions for the ERBACKMAP task (described below), which then generates a background map from a source-free image.

In [23]:
def run_erbox(image_file, exposure_map, detmask_file, output_boxlist, bkg_map=None, bg_image_flag="N", ecf=1, emin=200, emax=2300, log_file=None):
    if bg_image_flag=="N":
        subprocess.run(["erbox", 
                        f"images={image_file}", 
                        f"expimages={exposure_map}",
                        f"detmasks={detmask_file}",
                        f"boxlist={output_boxlist}",
                        f"emin={emin}",
                        f"emax={emax}",
                        f"bkgima_flag={bg_image_flag}",
                        f"ecf={ecf}",
                        ], stdout=log_file, stderr=log_file)
    else:
        subprocess.run(["erbox", 
                        f"images={image_file}", 
                        f"expimages={exposure_map}",
                        f"detmasks={detmask_file}",
                        f"boxlist={output_boxlist}",
                        f"emin={emin}",
                        f"emax={emax}",
                        f"bkgimages={bkg_map}",
                        f"ecf={ecf}",
                        ], stdout=log_file, stderr=log_file)

In [26]:
output_boxlist_local = "../Data/Source_cat/boxlist_local.fits"

if os.path.exists(output_boxlist_local):
    os.remove(output_boxlist_local)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbox(image_file, expmap_file, output_mask_file, 
              output_boxlist_local, log_file=log_file)

## ERBACKMAP

In [27]:
def run_erbackmap(image_file, exposure_map, detmask_file, boxlist_file, output_bkgmap, output_cheesemask, emin=200, emax=2300, log_file=None):
    subprocess.run(["erbackmap", 
                    f"image={image_file}", 
                    f"expimage={exposure_map}",
                    f"detmask={detmask_file}",
                    f"boxlist={boxlist_file}",
                    f"bkgimage={output_bkgmap}",
                    f"cheesemask={output_cheesemask}",
                    # f"emin={emin}",
                    f"emax={emax}",
                    "cheesemask_flag=Y",
                    "clobber=Y",
                    ], stdout=log_file, stderr=log_file)

In [28]:
output_bkgmap = "../Data/Source_cat/BG_maps/bkg_map.fits"
output_cheesemask = "../Data/Source_cat/BG_maps/cheesemask.fits"

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbackmap(image_file, expmap_file, output_mask_file, 
                  output_boxlist_local, output_bkgmap, output_cheesemask, 
                  log_file=log_file)

## ERBOX (map):

In [29]:
output_boxlist_map = "../Data/Source_cat/boxlist_map.fits"

if os.path.exists(output_boxlist_map):
    os.remove(output_boxlist_map)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbox(image_file, expmap_file, output_mask_file, 
              output_boxlist_map, output_bkgmap, log_file=log_file)

## ERMLDET:

In [30]:
def run_ermldet(image_file, exposure_map, detmask_file, boxlist_file,
                bkg_map, output_mllist, output_sourceimage, emin=200, emax=2300, log_file=None):
    subprocess.run(["ermldet", 
                    f"mllist={output_mllist}", 
                    f"boxlist={boxlist_file}",
                    f"images={image_file}",
                    f"expimages={exposure_map}",
                    f"detmasks={detmask_file}",
                    f"bkgimages={bkg_map}",
                    f"srcimages={output_sourceimage}",
                    "extentmodel=gaussian",
                    f"emin={emin}",
                    f"emax={emax}",
                    "srcima_flag=Y"
                    ], stdout=log_file, stderr=log_file)

In [32]:
output_mllist = "../Data/Source_cat/mllist.fits"
output_sourceimage = "../Data/Source_cat/sourceimage.fits"

if os.path.exists(output_mllist):
    os.remove(output_mllist)

if os.path.exists(output_sourceimage):
    os.remove(output_sourceimage)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_ermldet(image_file, expmap_file, output_mask_file, 
                output_boxlist_map, output_bkgmap, 
                output_mllist, output_sourceimage, log_file=log_file)

## CATPREP:

In [33]:
def run_catprep(input_mllist, out_catfile, log_file=None):
    subprocess.run(["catprep", 
                    f"infile={input_mllist}", 
                    f"outfile={out_catfile}"
                    ], stdout=log_file, stderr=log_file)

output_catalog = "../Data/Source_cat/catalog.fits"

if os.path.exists(output_catalog):
    os.remove(output_catalog)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_catprep(output_mllist, output_catalog, log_file=log_file)